# AHP
---
_**A**nalytic **H**ierarchy **P**rocess_

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

## 1. Introduction
---
The **AHP** (_Analytic Hierarchy Process_) method is a structured technique for calculating priority weights for multiple criteria, in order to support decisions. The _AHP_ hierarchy structures a complex problem into an orientation of **objectives**, **criteria**/**subcriteria** and **alternatives**.

![](https://upload.wikimedia.org/wikipedia/commons/d/db/AHPHierarchy1Labeled.png)

_Depending on the complexity of the problem, the model may use sub-criteria._

### 1.1. Applications
---
The _AHP method_ can be used in a variety of dicision situations where priorization is the thing, such as:

- Supplier selection
- Assessment of job candidates
- Choice of products
- Feature priorization

For example, let's consider a set of elements with **4 criteria** and **5 alternatives**, generated randomly:

In [3]:
np.random.seed(0)

M = 5
C = np.random.randint(100, 500, M)
df = pd.DataFrame({
    "Criteria 1": C,
    "Criteria 2": (C**(np.random.random(M)/2)).astype(int),
    "Criteria 3": np.log(C).astype(int),
    "Criteria 4": (C*(1 + np.random.random(M))//2).astype(int)
})

df.index = [f'Alternative {i}' for i in np.arange(M)+1]

df.style.background_gradient()

,Criteria 1,Criteria 2,Criteria 3,Criteria 4
Alternative 1,272,11,5,143
Alternative 2,147,8,4,93
Alternative 3,217,5,5,160
Alternative 4,292,2,5,264
Alternative 5,423,2,6,313


## 2. Method steps
---

### 2.1. Pairwise comparison matrix
---
The first step is to create the _pairwise comparison matrix_ for all criteria, and its values are determinated based on a **scale of relative importance**:


|       scale        | importance           |
| :----------------: | :------------------- |
|         1          | Equally preferred    |
|         3          | Mildly preferred     |
|         5          | Moderately preferred |
|         7          | Greatly preferred    |
|         9          | Always preferred     |
|     2, 4, 6, 8     | Intermediate         |
| 1/3, 1/5, 1/7, 1/9 | Inverse comparison   |

The comparison is made pairwise between the variables and the inverse value must be used for a reciprocal ($a_{ij} = \frac{1}{a_{ji}}$) action. For example, consider three criteria **A**, **B** and **C**, their matrix would be:

$$ \large
M =
\begin{bmatrix}
    1 & A_B & A_C \\
    B_A & 1 & B_C \\
    C_A & C_B & 1
\end{bmatrix} = 
\begin{bmatrix}
    1 & A_B & A_C \\
    \frac{1}{A_B} & 1 & B_C \\
    \frac{1}{A_C} & \frac{1}{B_C} & 1
\end{bmatrix}
$$

This construction assumes that if $A > B$ and $B > C$ then $A > C$.

In [4]:
PWCM = pd.DataFrame([
    [1  , 5  , 4  , 7],
    [1/5, 1  , 1/2, 3],
    [1/4, 2  , 1  , 3],
    [1/7, 1/3, 1/3, 1]
]).astype(float)

PWCM.index = df.columns
PWCM.columns = df.columns

PWCM

,Criteria 1,Criteria 2,Criteria 3,Criteria 4
Criteria 1,1.000000,5.000000,4.000000,7.0
Criteria 2,0.200000,1.000000,0.500000,3.0
Criteria 3,0.250000,2.000000,1.000000,3.0
Criteria 4,0.142857,0.333333,0.333333,1.0


### 2.2. Normalize pairwise comparizon matrix
---
The second step is the matrix normalization, which is done by dividing the element by the sum of its column.

$$ \large
a_{ij}' = \frac{a_{ij}}{\sum_{i}^m a_{ij}} 
$$

In [5]:
PWCM_N = PWCM/PWCM.sum(axis=0)

PWCM_N

,Criteria 1,Criteria 2,Criteria 3,Criteria 4
Criteria 1,0.627803,0.60,0.685714,0.500000
Criteria 2,0.125561,0.12,0.085714,0.214286
Criteria 3,0.156951,0.24,0.171429,0.214286
Criteria 4,0.089686,0.04,0.057143,0.071429


### 2.3. Criteria weights
---
The third step is the definition of the criteria weights, which is done by calculating the mean value of each normalized row.

$$ \large
w_i = \frac{1}{n} \sum_{j}^n a_{ij}
$$

In [6]:
CRITERIA_WEIGHTS = PWCM_N.mean(axis=1)

CRITERIA_WEIGHTS

Criteria 1    0.603379
Criteria 2    0.136390
Criteria 3    0.195666
Criteria 4    0.064564
dtype: float64

### 2.4. Consistency ratio
---
The fourth step is the calculation of the _consistency ratio_, which tell us if the pairwise comparison matrix was constructed consistently. For this, we have firstly to calculate the weighted weights, multiplying the pairwise comparison matrix by the criteria weights and summing all rows.

$$ \large
w_i' = \sum_j^n M_{ij} \cdot w_j
$$

Secondly, we have to find the $\large \lambda_{max}$.

$$ \large
\lambda_{max} = \frac{1}{n} \sum \frac{w'}{w}
$$

In [7]:
PWCM_CW = PWCM*CRITERIA_WEIGHTS
PWCM_CW = PWCM_CW.sum(axis=1)
PWCM_CW = PWCM_CW/CRITERIA_WEIGHTS

lambda_max = PWCM_CW.mean()

lambda_max

4.100742229245654

After, we need the **consistency index**.

$$ \large
CI = \frac{\lambda_{max} - n}{n - 1}
$$

In [8]:
n = len(CRITERIA_WEIGHTS)
CI = (lambda_max - n)/(n - 1)

CI

0.033580743081884634

Finally, we calculate the **consistency ratio**.

$$ \large
CR = \frac{CI}{RI}
$$

where $RI$ is an index of consistence of an random matrix, which depends on the order of this matrix.

| order |  1  |  2  |  3   |  4  |  5   |  6   |  7   |  8   |  9   |  10  |
| :---: | :-: | :-: | :--: | :-: | :--: | :--: | :--: | :--: | :--: | :--: |
|  RI   |  0  |  0  | 0.58 | 0.9 | 1.12 | 1.24 | 1.32 | 1.41 | 1.45 | 1.49 |

For CR to be considered consistent, it must be less than 10%.

In [9]:
RI = {
    1: 0,
    2: 0,
    3: 0.58,
    4: 0.90,
    5: 1.12,
    6: 1.24,
    7: 1.32,
    8: 1.41,
    9: 1.45,
    10: 1.49
}[n]

CR = CI/RI

print(f'''
Consistency ratio: {CR:.04f}
    Less than 10%: {CR < 0.1}
''')


Consistency ratio: 0.0373
    Less than 10%: True



### 2.5. Score and Ranking
---
The last step, after having verified the _criteria weights' consistency ratio_, is to apply it to the alternatives data and build a **score**. Firstly, we have to check the criteria we want to **minimize** instead of **maximize** and transform them before proceeding.

$$ \large
\min C_j \quad \therefore \quad a_{ij}' = \frac{1}{a_{ij}}
$$

Finally, we can calculate the score over the transformed data.

$$ \large
S_i = \sum_j^n \left[ \frac{a_{ij}'}{\sum_{i}^m a_{ij}'} \cdot w_j \right]
$$

Having that, we are able to define a **ranking** by ordering the score in descending way.

In [10]:
df_a = df.copy()

df_a["Criteria 2"] = 1/df_a["Criteria 2"]
df_a = df_a/df_a.sum()

df["score"] = np.sum(df_a*CRITERIA_WEIGHTS, axis=1)
df["ranking"] = (
    df["score"]
    .rank(ascending=False, method="first")
    .astype(int)
)

(
    df.sort_values("ranking")
    .style.background_gradient(
        subset=list(df_a.columns)
    )
)

,Criteria 1,Criteria 2,Criteria 3,Criteria 4,score,ranking
Alternative 5,423,2,6,313,0.304812,1
Alternative 4,292,2,5,264,0.235227,2
Alternative 1,272,11,5,143,0.178859,3
Alternative 3,217,5,5,160,0.165931,4
Alternative 2,147,8,4,93,0.115171,5


## 3. Examples
---

### 3.1. Device priorization
---
As an example, let's priorize devices based on their attributes. The data was taken from [dxomark](https://www.dxomark.com/smartphones/) and we will consider 5 criteria: **price**, **camera**, **display** and **battery**. The _price_ is a value we want to **minimize** so we have to prepare this feature before calculating the score.

In [11]:
df_ex1 = pd.DataFrame({
    # criteria we want to minimize
    "price": [1199, 999, 1299, 1199, 908],
    # criteria we want to maximize
    "camera": [154, 153, 149, 140, 140],
    "display": [151, 154, 133, 148, 130],
    "battery": [134, 111, 103, 142, 133]
})

df_ex1.index = [
    "Apple iPhone 15 Pro Max", 
    "Google Pixel 8 Pro",
    "Huawei Mate 50 Pro",
    "Samsung Galaxy S23 Ultra",
    "Xiaomi 13 Ultra",
]

df_ex1.style.background_gradient()

,price,camera,display,battery
Apple iPhone 15 Pro Max,1199,154,151,134
Google Pixel 8 Pro,999,153,154,111
Huawei Mate 50 Pro,1299,149,133,103
Samsung Galaxy S23 Ultra,1199,140,148,142
Xiaomi 13 Ultra,908,140,130,133


In [12]:
# Pairwise comparison matrix
# price > camera > display > battery
PWCM_ex1 = pd.DataFrame([
    [1  , 3  , 7  , 5],
    [1/3, 1  , 4  , 2],
    [1/7, 1/4, 1  , 2],
    [1/5, 1/2, 1/2, 1]
]).astype(float)

PWCM_ex1.index = df_ex1.columns
PWCM_ex1.columns = df_ex1.columns

# Normalize pairwise comparizon matrix
PWCM_ex1_N = PWCM_ex1/PWCM_ex1.sum(axis=0)

# Criteria weights
CRITERIA_WEIGHTS = PWCM_ex1_N.mean(axis=1)

# Consistency ratio
PWCM_ex1_CW = PWCM_ex1*CRITERIA_WEIGHTS
PWCM_ex1_CW = PWCM_ex1_CW.sum(axis=1)
PWCM_ex1_CW = PWCM_ex1_CW/CRITERIA_WEIGHTS

lambda_max = PWCM_ex1_CW.mean()

n = len(CRITERIA_WEIGHTS)
CI = (lambda_max - n)/(n - 1)

RI = {
    1: 0,
    2: 0,
    3: 0.58,
    4: 0.90,
    5: 1.12,
    6: 1.24,
    7: 1.32,
    8: 1.41,
    9: 1.45,
    10: 1.49
}[n]

CR = CI/RI

if not CR < 0.1:
    raise Exception("Consistency ratio not acceptable!")

# Application
df_ex1_a = df_ex1.copy()

df_ex1_a["price"] = 1/df_ex1_a["price"]
df_ex1_a = df_ex1_a/df_ex1_a.sum()

df_ex1["score"] = np.sum(df_ex1_a*CRITERIA_WEIGHTS, axis=1)
df_ex1["ranking"] = (
    df_ex1["score"]
    .rank(ascending=False, method="first")
    .astype(int)
)

(
    df_ex1.sort_values("ranking")
    .style.background_gradient(
        subset=list(df_ex1_a.columns)
    )
)

,price,camera,display,battery,score,ranking
Xiaomi 13 Ultra,908,140,130,133,0.221383,1
Google Pixel 8 Pro,999,153,154,111,0.213130,2
Apple iPhone 15 Pro Max,1199,154,151,134,0.195335,3
Samsung Galaxy S23 Ultra,1199,140,148,142,0.191648,4
Huawei Mate 50 Pro,1299,149,133,103,0.178505,5
